# Neeps - Hard

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app15-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@77912764

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@16f97e43

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val ut_staff = hiveCxt.table("sqlzoo.ut_staff")
val ut_student = hiveCxt.table("sqlzoo.ut_student")
val ut_event = hiveCxt.table("sqlzoo.ut_event")
val ut_room = hiveCxt.table("sqlzoo.ut_room")
val ut_attends = hiveCxt.table("sqlzoo.ut_attends")
val ut_teaches = hiveCxt.table("sqlzoo.ut_teaches")
val ut_occurs = hiveCxt.table("sqlzoo.ut_occurs")
val ut_modle = hiveCxt.table("sqlzoo.ut_modle")
val ut_week = hiveCxt.table("sqlzoo.ut_week")

ut_staff: DataFrame = [id: string, name: string]
ut_student: DataFrame = [id: string, name: string ... 2 more fields]
ut_event: DataFrame = [id: string, modle: string ... 5 more fields]
ut_room: DataFrame = [id: string, name: string ... 2 more fields]
ut_attends: DataFrame = [student: string, event: string]
ut_teaches: DataFrame = [staff: string, event: string]
ut_occurs: DataFrame = [event: string, week: string]
ut_modle: DataFrame = [id: string, name: string]
ut_week: DataFrame = [id: string, wkstart: string]

## 11.
**co.CHt is to be given all the teaching that co.ACg currently does. Identify those events which will clash.**

In [5]:
val t = (ut_event.withColumnRenamed("id", "event")
     .join(ut_teaches, "event")
     .join(ut_occurs, "event")
     .withColumn("begin", to_timestamp(col("tod"), "HH:mm"))
     .withColumn("end", (unix_timestamp(col("tod"), "HH:mm") + 
                 (lit(3600) * col("duration"))).cast("timestamp")))

(t.filter(col("staff") === "co.CHt").alias("t1")
 .join(t.filter(col("staff") === "co.ACg").alias("t2"), 
       Seq("week", "dow"))
 .filter(((col("t1.begin") >= col("t2.begin")) && 
          (col("t1.begin") < col("t2.end"))) ||
         ((col("t2.begin") >= col("t1.begin")) && 
          (col("t2.begin") < col("t1.end"))))
 .select(col("t1.event").alias("co.Cht"), 
         col("t2.event").alias("co.ACg"))
 .distinct()
 .showHTML())

co.Cht,co.ACg
co12005.T03,co12005.T01
co12005.T03,co72013.L02
co12005.T03,co72013.T03


t: DataFrame = [event: string, modle: string ... 9 more fields]

## 12.
**Produce a table showing the utilisation rate and the occupancy level for all rooms with a capacity more than 60.**

> I don't know how "utilisation rate" and "occupancy level" are defined.

In [6]:
// theoretical full utilisation hours: 15 weeks * 5 days * 12 hours
val oc_time = (
    ut_event
       .withColumn("close_time", 
                   (unix_timestamp(col("tod"), "HH:mm") + 
                   lit(3600) * col("duration")).cast("timestamp"))
       .withColumn("open_time", 
                   to_timestamp(col("tod"), "HH:mm"))
      .agg(max("close_time"), min("open_time"))
      .withColumn("max_hrs", ($"max(close_time)"-$"min(open_time)")
                  .cast("long")/lit(3600)))

val t = (ut_week.join(oc_time.select("max_hrs"))
         .withColumn("max_hrs", lit(5) * $"max_hrs".cast("long")))
t.show()

// attended students per event
val s = (ut_event.withColumnRenamed("id", "event")
     .join(ut_occurs, "event")
     .join(ut_attends, "event")
     .join(ut_student, (ut_attends("student") === ut_student("id")))
     .join(ut_room, (ut_event("room") === ut_room("id")))
     .groupBy("room")
     .agg(sum("sze"), sum("capacity")))

(ut_event.withColumnRenamed("id", "event")
 .join(ut_occurs, "event")
 .join(ut_room.filter(col("capacity") > 60), 
       (ut_event("room") === ut_room("id")))
 .groupBy("room")
 .sum("duration")
 .join(s, "room")
 .withColumn("util", col("sum(duration)") /
             t.groupBy().sum("max_hrs").first.getLong(0))
 .withColumn("occup_level", col("sum(sze)")/col("sum(capacity)"))
 .select("room", "util", "occup_level")
 .showHTML())

+---+----------+-------+
| id|   wkstart|max_hrs|
+---+----------+-------+
| 01|2018-10-01|     60|
| 02|2018-10-08|     60|
| 03|2018-10-15|     60|
| 04|2018-10-22|     60|
| 05|2018-10-29|     60|
| 06|2018-11-05|     60|
| 07|2018-11-12|     60|
| 08|2018-11-19|     60|
| 09|2018-11-26|     60|
| 10|2018-12-03|     60|
| 11|2018-12-10|     60|
| 12|2018-12-17|     60|
| 13|2019-01-07|     60|
| 14|2019-01-14|     60|
| 15|2019-01-21|     60|
+---+----------+-------+



room,util,occup_level
cr.SMH,0.24,0.2745744680851064


oc_time: DataFrame = [max(close_time): timestamp, min(open_time): timestamp ... 1 more field]
t: DataFrame = [id: string, wkstart: string ... 1 more field]
s: DataFrame = [room: string, sum(sze): bigint ... 1 more field]

## 13.
**A one hour staff meeting is to be held between 09:00 and 17:00. Events which clash are to be cancelled. Identify the hour which will result in the least disruption.**

In [7]:
val t = (ut_event.withColumn("tod", to_timestamp(ut_event("tod"), "HH:mm"))
         .withColumn("duration", ut_event("duration").cast("integer"))
         .join(ut_occurs, (ut_event("id") === ut_occurs("event")))
         .select("event", "week", "dow", "tod", "duration"))

var dfs = Seq[DataFrame]()
for (i <- 1 to 10){
    val t_ = t.filter(t("duration") === i)
    if (t_.count() > 0) {
        val t__ = t_.withColumn("tod", t_("tod") + expr(
            "interval " + (i-1).toString + " hours"))
        dfs = dfs :+ t__
    }
}
val df = dfs.reduce(_.union(_))

(df.na.replace("dow", Map("Monday" -> "1", "Tuesday" -> "2", "Wednesday" -> "3", 
                          "Thursday" -> "4", "Friday" -> "5"))
 .withColumn("hour", hour(col("tod")))
 .groupBy("dow", "hour")
 .pivot("week")
 .count()
 .orderBy("dow", "hour")
 .showHTML())

dow,hour,01,02,03,04,05,06,07,08,10,11,12,13
1,9,1,1,1,1,1,1,1,1,1,1,1,1
1,10,3,3,3,3,3,3,3,3,3,3,3,3
1,11,1,1,1,1,1,1,1,1,1,1,1,1
1,12,7,7,7,7,7,7,7,7,7,7,7,7
1,13,2,2,2,2,2,2,2,2,2,2,2,2
1,14,5,5,5,5,5,5,5,5,5,5,5,5
1,15,3,3,3,3,3,3,3,3,3,3,3,3
1,16,8,8,8,8,8,8,8,8,8,8,8,8
1,17,2,2,2,2,2,2,2,2,2,2,2,2
2,9,2,2,2,2,2,2,2,2,2,2,2,2


t: DataFrame = [event: string, week: string ... 3 more fields]
dfs: Seq[DataFrame] = List(
  [event: string, week: string ... 3 more fields],
  [event: string, week: string ... 3 more fields]
)
df: DataFrame = [event: string, week: string ... 3 more fields]

In [8]:
// You may find that week "09", "14", "15" are totally vacant
// or you can choose Mon 9am/12pm, Tue 5pm, Thu 9am, Fri 5pm

(df.withColumn("tod", date_format($"tod", "HH:mm"))
 .filter(hour(col("tod")).between(9, 17))
 .groupBy("tod", "dow")
 .agg(count("event").alias("event"))
 .orderBy("event", "tod", "dow")
 .limit(1)
 .showHTML())

tod,dow,event
09:00,Monday,12


## 14.
**Find all clashes - include the events which clash and the staff, student or rooms that they have in common.**

In [9]:
val t = (ut_event
     .withColumn("start", to_timestamp(ut_event("tod"), "HH:mm"))
     .withColumnRenamed("id", "event")
     .join(ut_occurs, "event")
     .withColumn("end", (unix_timestamp(col("start")) + lit(3600)*col("duration"))
                 .cast("timestamp")))
(t.alias("t1")
 .join(t.alias("t2"), Seq("week", "dow", "room"))
 .filter(col("t1.event") !== col("t2.event"))
 .filter((((col("t1.start") >= col("t2.start")) &&
           (col("t1.start") < col("t2.end"))) ||
          ((col("t2.start") >= col("t1.start")) &&
           (col("t2.start") < col("t1.end")))) &&
         (col("t1.event") > col("t2.event")))
 .select(col("t1.event").alias("event1"),
         col("t2.event").alias("event2"),
         col("room"), col("week"), col("dow"))
 .distinct()
 .showHTML())

event1,event2,room,week,dow
co22020.T04,co22007.T01,co.LB47,01,Friday
coh8412615.T03,co22008.T04,co.117+118,11,Tuesday
co22020.T04,co22007.T01,co.LB47,12,Friday
coh8412615.T03,co22008.T04,co.117+118,02,Tuesday
co22020.T04,co22007.T01,co.LB47,02,Friday
co22020.T04,co22007.T01,co.LB47,07,Friday
coh8412615.T03,co22008.T04,co.117+118,08,Tuesday
coh8412615.T03,co22008.T04,co.117+118,03,Tuesday
co22020.T04,co22007.T01,co.LB47,06,Friday
coh8412615.T03,co22008.T04,co.117+118,10,Tuesday


t: DataFrame = [event: string, modle: string ... 8 more fields]

## 15.
**Produce a timetable for a group of full time students for week 1**

In [10]:
val t = (ut_event
     .withColumn("tod", to_timestamp(ut_event("tod"), "HH:mm"))
     .withColumnRenamed("id", "event")
     .join(ut_occurs.filter(col("week") === "01"), "event")
     .join(ut_attends, "event"))

var dfs = Seq[DataFrame]()
for (i <- 1 to 10){
    val t_ = t.filter(t("duration") === i)
    if (t_.count() > 0){
        val t__ = t_.withColumn("tod", t_("tod") + expr(
            "interval " + (i-1).toString + " hours"))
        dfs = dfs :+ t__
    }
}
val df = dfs.reduce(_.union(_))

(df.withColumn("tod", hour(col("tod")))
 .withColumn("flag", lit("Y"))
 .groupBy("dow", "student")
 .pivot("tod")
 .agg(first("flag"))
 .na.fill("")
 .showHTML())

dow,student,9,10,11,12,13,14,15,16,17,19,20
Friday,co.12008.Eb,,,,,,,Y,,,,
Friday,co.22022.E,Y,,Y,,,,,,,,
Friday,co1.CO,,Y,,,,,,,,,
Friday,co2.CO,,,,,,,,Y,,,
Friday,co2.CO.a,Y,,,Y,,,,,,,
Friday,co2.CO.b,Y,,,,,,,,,,
Friday,co2.CO.e,,Y,,Y,,,,,,,
Friday,co2.NT.Ea,,,,,,Y,,,,,
Friday,co2.NT.Eb,,,,,,Y,,Y,,,
Friday,co2.NT.Ec,,,,,,Y,,,,,


t: DataFrame = [event: string, modle: string ... 7 more fields]
dfs: Seq[DataFrame] = List(
  [event: string, modle: string ... 7 more fields],
  [event: string, modle: string ... 7 more fields]
)
df: DataFrame = [event: string, modle: string ... 7 more fields]

In [11]:
spark.stop()